# GROUP FINAL PROJECT-  PREDICTING TOP PLAYERS FOR 2021 

In [68]:
#Import the various tools that we will need as we plot our different types of graphs.  I am commenting these out as this assignmment does not require visualizations
# %matplotlib inline
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
import pandas as pd
import numpy as np
# import plotly.graph_objects as go

## Import the data from GitHub

In [69]:
baseballstats = pd.read_csv("https://raw.githubusercontent.com/zoberender/Python-Group-Project/master/Batting.csv")
#It is important to convert NaN to 0 to get calculations to function correctly
baseballstats['IBB'] = baseballstats['IBB'].fillna(0)
baseballstats['HBP'] = baseballstats['HBP'].fillna(0)
baseballstats['SH'] = baseballstats['SH'].fillna(0)
baseballstats['SF'] = baseballstats['SF'].fillna(0)
# Create new column to calculate average
baseballstats["AVG"] = baseballstats["H"]/baseballstats["AB"]
# Create new column to calculate total bases
baseballstats["TB"] = baseballstats["H"] - baseballstats["2B"] - baseballstats["3B"] - baseballstats["HR"] + 2*baseballstats["2B"] + 3*baseballstats["3B"]+4*baseballstats["HR"]
# Slugging is Total Bases/At bats
baseballstats["SLG"] = baseballstats["TB"]/baseballstats["AB"]
#Plate Appearances to calculate OBP
baseballstats["PA"] = baseballstats["AB"] + baseballstats["BB"] + baseballstats["HBP"] + baseballstats["SF"]
#On Base Percentage = Times on base/Plat Appearances
baseballstats["OBP"] = (baseballstats["H"]+baseballstats["BB"]+baseballstats["HBP"])/baseballstats["PA"]
#OBP + SLUGGING IS A GREAT WAY TO MEASURE OVERALL PLAYER VALUE
baseballstats["OPS"] = baseballstats["OBP"] + baseballstats["SLG"]
#ISOLATED POWER IS A WAY TO MEASURE A PLAYER'S OVERALL POWER WHY TAKING OUT THE IMPACT OF BATTING AVERAGE
baseballstats['ISP'] = baseballstats['SLG'] - baseballstats['AVG']
#Add 1B as its own column.  1B = HR - (2B + 3B + HR)
baseballstats["1B"] = baseballstats["H"] - baseballstats["2B"] - baseballstats["3B"] - baseballstats["HR"]
#Add ERA to the data fram using by binning
cut_label = ['19th Century','Dead Ball','Live Ball','Integration','Divisional','Steroid','Current']
cut_bins = [0,1899,1919,1946,1968,1993,2007,max(baseballstats['yearID'])]
baseballstats['ERA'] = pd.cut(baseballstats['yearID'], bins=cut_bins, labels = cut_label)
#Filter for Plate Appearances greater than 200 to eliminate the lower volume plate appearances.
baseballstats = baseballstats[(baseballstats["PA"] >= 200)]

baseballstats

,playerID,yearID,Stint,teamID,lgID,G,AB,R,H,2B,...,TEAM NAME,AVG,TB,SLG,PA,OBP,OPS,ISP,1B,ERA
118,ansonca01,1872,1,PH1,NaN,46,217,60,90,10,...,PNA-Philadelphia Athletics,0.414747,114,0.525346,233.0,0.454936,0.980281,0.110599,73,19th Century
121,barnero01,1872,1,BS1,NaN,45,230,81,99,28,...,BNA-Boston Red Stockings,0.430435,134,0.582609,239.0,0.451883,1.034492,0.152174,68,19th Century
125,bechtge01,1872,1,NY2,NaN,51,247,61,74,11,...,NNA-New York Mutuals,0.299595,91,0.368421,254.0,0.318898,0.687319,0.068826,60,19th Century
144,cummica01,1872,1,NY2,NaN,55,249,37,52,9,...,NNA-New York Mutuals,0.208835,67,0.269076,253.0,0.221344,0.490420,0.060241,40,19th Century
145,cuthbne01,1872,1,PH1,NaN,47,260,83,88,10,...,PNA-Philadelphia Athletics,0.338462,101,0.388462,266.0,0.353383,0.741845,0.050000,77,19th Century
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107243,wolteto01,2019,1,COL,NL,121,359,42,94,17,...,COL-Colorado Rockies,0.261838,118,0.328691,409.0,0.337408,0.666099,0.066852,74,Current
107246,wongko01,2019,1,SLN,NL,148,478,61,136,25,...,STL-St. Louis Cardinals,0.284519,202,0.422594,543.0,0.360958,0.783552,0.138075,96,Current
107262,yastrmi01,2019,1,SFN,NL,107,371,64,101,22,...,SFG-San Francisco Giants,0.272237,192,0.517520,410.0,0.334146,0.851667,0.245283,55,Current
107264,yelicch01,2019,1,MIL,NL,130,489,100,161,29,...,MIL-Milwaukee Brewers,0.329243,328,0.670757,580.0,0.429310,1.100067,0.341513,85,Current


# Prepare the data

In [70]:
baseballstats.shape #This shows number of rows and columns

(30669, 34)

In [71]:
baseballstats.info() #This provides the number of non-null values for each category, as well as Data Type.  Must convert objects to integers

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30669 entries, 118 to 107277
Data columns (total 34 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   playerID   30669 non-null  object  
 1   yearID     30669 non-null  int64   
 2   Stint      30669 non-null  int64   
 3   teamID     30669 non-null  object  
 4   lgID       30477 non-null  object  
 5   G          30669 non-null  int64   
 6   AB         30669 non-null  int64   
 7   R          30669 non-null  int64   
 8   H          30669 non-null  int64   
 9   2B         30669 non-null  int64   
 10  3B         30669 non-null  int64   
 11  HR         30669 non-null  int64   
 12  RBI        30480 non-null  float64 
 13  SB         29716 non-null  float64 
 14  CS         22736 non-null  float64 
 15  BB         30669 non-null  int64   
 16  SO         30027 non-null  float64 
 17  IBB        30669 non-null  float64 
 18  HBP        30669 non-null  float64 
 19  SH         30669 non-n

In [72]:
baseballstats['OPS'].unique()

array([0.98028124, 1.03449154, 0.68731869, ..., 0.85166656, 1.10006699,
       0.54386399])

In [73]:
baseballstats['ERA'].unique()

['19th Century', 'Dead Ball', 'Live Ball', 'Integration', 'Divisional', 'Steroid', 'Current']
Categories (7, object): ['19th Century' < 'Dead Ball' < 'Live Ball' < 'Integration' < 'Divisional' < 'Steroid' < 'Current']

In [74]:
# Include columns of interest on this line
columns = ["OPS","lgID", "AGE", "PA", "ISP", "ERA"]
columns

['OPS', 'lgID', 'AGE', 'PA', 'ISP', 'ERA']

In [75]:
# Get the columns we actually care about
baseballstats = baseballstats[columns]
# Pay attention to the number of non-null counts.
baseballstats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30669 entries, 118 to 107277
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   OPS     30669 non-null  float64 
 1   lgID    30477 non-null  object  
 2   AGE     30669 non-null  int64   
 3   PA      30669 non-null  float64 
 4   ISP     30669 non-null  float64 
 5   ERA     30669 non-null  category
dtypes: category(1), float64(3), int64(1), object(1)
memory usage: 1.4+ MB


In [76]:
#Drop out the NA values
baseballstats.dropna(inplace=True)
baseballstats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30477 entries, 685 to 107277
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   OPS     30477 non-null  float64 
 1   lgID    30477 non-null  object  
 2   AGE     30477 non-null  int64   
 3   PA      30477 non-null  float64 
 4   ISP     30477 non-null  float64 
 5   ERA     30477 non-null  category
dtypes: category(1), float64(3), int64(1), object(1)
memory usage: 1.4+ MB


<ipython-input-76-5fce26f8aa37>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseballstats.dropna(inplace=True)


In [77]:
#Get dummy variables for league
pd.get_dummies(baseballstats[["lgID"]], prefix_sep= "::", drop_first = True)

,lgID::AL,lgID::FL,lgID::NL,lgID::PL,lgID::UA
685,0,0,1,0,0
686,0,0,1,0,0
687,0,0,1,0,0
692,0,0,1,0,0
694,0,0,1,0,0
...,...,...,...,...,...
107243,0,0,1,0,0
107246,0,0,1,0,0
107262,0,0,1,0,0
107264,0,0,1,0,0


In [78]:
# Get dummy variables for Era
pd.get_dummies(baseballstats[["ERA"]], prefix_sep= "::", drop_first = True)

,ERA::Dead Ball,ERA::Live Ball,ERA::Integration,ERA::Divisional,ERA::Steroid,ERA::Current
685,0,0,0,0,0,0
686,0,0,0,0,0,0
687,0,0,0,0,0,0
692,0,0,0,0,0,0
694,0,0,0,0,0,0
...,...,...,...,...,...,...
107243,0,0,0,0,0,1
107246,0,0,0,0,0,1
107262,0,0,0,0,0,1
107264,0,0,0,0,0,1


In [79]:
# Helper function to colosidate information
def createCategoricalDummies(dataFrame, categoryList):
    return pd.get_dummies(dataFrame[categoryList], prefix_sep = "::", drop_first = True)

In [80]:
# Combine tables- numerical with dummy variables
categories = ["lgID", "ERA"]

baseballstats = pd.concat(
    [baseballstats.drop(categories, axis=1), createCategoricalDummies(baseballstats, categories)], axis= 1)
baseballstats

,OPS,AGE,PA,ISP,lgID::AL,lgID::FL,lgID::NL,lgID::PL,lgID::UA,ERA::Dead Ball,ERA::Live Ball,ERA::Integration,ERA::Divisional,ERA::Steroid,ERA::Current
685,0.829900,24,321.0,0.093851,0,0,1,0,0,0,0,0,0,0,0
686,1.052050,26,342.0,0.161491,0,0,1,0,0,0,0,0,0,0,0
687,0.682370,22,289.0,0.067138,0,0,1,0,0,0,0,0,0,0,0
692,0.532268,22,266.0,0.056818,0,0,1,0,0,0,0,0,0,0,0
694,0.556756,27,281.0,0.011029,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107243,0.666099,27,409.0,0.066852,0,0,1,0,0,0,0,0,0,0,1
107246,0.783552,28,543.0,0.138075,0,0,1,0,0,0,0,0,0,0,1
107262,0.851667,28,410.0,0.245283,0,0,1,0,0,0,0,0,0,0,1
107264,1.100067,27,580.0,0.341513,0,0,1,0,0,0,0,0,0,0,1


# Import Machine Learning Libraries for Regression and get testing and training data

In [81]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression

In [82]:
features = list(baseballstats.columns)
features.remove("OPS")
target = "OPS"

print(f"Feature categories: {features}")
print(f"Target feature: {target}")

Feature categories: ['AGE', 'PA', 'ISP', 'lgID::AL', 'lgID::FL', 'lgID::NL', 'lgID::PL', 'lgID::UA', 'ERA::Dead Ball', 'ERA::Live Ball', 'ERA::Integration', 'ERA::Divisional', 'ERA::Steroid', 'ERA::Current']
Target feature: OPS


In [83]:
PredInput = baseballstats[features]
PredInput

,AGE,PA,ISP,lgID::AL,lgID::FL,lgID::NL,lgID::PL,lgID::UA,ERA::Dead Ball,ERA::Live Ball,ERA::Integration,ERA::Divisional,ERA::Steroid,ERA::Current
685,24,321.0,0.093851,0,0,1,0,0,0,0,0,0,0,0
686,26,342.0,0.161491,0,0,1,0,0,0,0,0,0,0,0
687,22,289.0,0.067138,0,0,1,0,0,0,0,0,0,0,0
692,22,266.0,0.056818,0,0,1,0,0,0,0,0,0,0,0
694,27,281.0,0.011029,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107243,27,409.0,0.066852,0,0,1,0,0,0,0,0,0,0,1
107246,28,543.0,0.138075,0,0,1,0,0,0,0,0,0,0,1
107262,28,410.0,0.245283,0,0,1,0,0,0,0,0,0,0,1
107264,27,580.0,0.341513,0,0,1,0,0,0,0,0,0,0,1


In [84]:
OPSTarget = baseballstats[target]
OPSTarget

685       0.829900
686       1.052050
687       0.682370
692       0.532268
694       0.556756
            ...   
107243    0.666099
107246    0.783552
107262    0.851667
107264    1.100067
107277    0.543864
Name: OPS, Length: 30477, dtype: float64

In [85]:
# Create test and train data

X_train, X_test, y_train, y_test = train_test_split(PredInput,OPSTarget, test_size=0.25)

print(f"Length of X_train (feature training set): {len(X_train)}")
print(f"Length of X_test (feature test set): {len(X_test)}")
print(f"Length of y_train (target training set): {len(y_train)}")
print(f"Length of y_test (target training set): {len(y_test)}")

Length of X_train (feature training set): 22857
Length of X_test (feature test set): 7620
Length of y_train (target training set): 22857
Length of y_test (target training set): 7620


# Linear Regression Model
##### NOTE:  Linear Regression demonstrated a higher R-Squared value than Ridge Regression, so it is the model I will choose for this analysis

In [86]:
lr = LinearRegression()
lr

LinearRegression()

In [87]:
lr.fit(X_train, y_train)

LinearRegression()

In [88]:
lr.score(X_train, y_train)  # What's our score with the training data set?

0.7086277112530093

In [89]:
lr.score(X_test, y_test)    # What's our score with the test data set?

0.7159343112911649

In [90]:
import math
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score, mean_squared_error

def printMetrics(test, predictions):
    print(f"Score: {explained_variance_score(test, predictions):.2f}")
    print(f"MAE: {mean_absolute_error(test, predictions):.2f}")
    print(f"RMSE: {math.sqrt(mean_squared_error(test, predictions)):.2f}")
    print(f"r2: {r2_score(test, predictions):.2f}")

In [91]:
predictions = lr.predict(X_test)
printMetrics(y_test, predictions)

Score: 0.72
MAE: 0.05
RMSE: 0.06
r2: 0.72


In [92]:
import random as rnd
rnd.seed(1024)

In [93]:
# This would be a good way to test out random values for unknown data.  Being that our data is known, I am entering the known values.  
# numElements = 3
# samplebbstats = []
# for _ in range(numElements):
#    dict = {}
#    for column in PredInput.columns:
#        min = 0  # We'll always allow at lea
#        maxValue = round(max(baseballstats[column].values))
#        dict[column] = rnd.randint(min, maxValue)
#    samplebbstats.append(dict)
# samplebbstats

#####  The 10 highest rated Baseball Players are listed below, and I want to see which one I expect to have the highest OPS next season.  This will help me in my Fantasy Baseball League.  A couple of assumptions-  I am assuming that the player stays as healthy as in previous seasons with similiar plate appearances.  I am also assuming that Isolated Power is similar to other seasons as well.
#####  Freddie Freeman is 31 years old.  He plays for the Atlanta Braves in the NL.  His 162 game averages are 687 plate appearances and an ISP of 0.214
#####  Mike Trout is 29 years old.  He plays for the Los Angeles Angels in the AL.  His 162 game averages are 712 plate appearances and an ISP of 0.278
#####  Mookie Betts is 28 years old.  He plays for the Los Angeles Dodgers in the NL.  His 162 game averages are 739 plate appearances and an ISP of 0.221
#####  Ronald Acuna is 23 years old. He plays for the Atlanta Braves in the NL.  His 162 game averages are 727 plate appearances and an ISP of 0.257
#####  Fernando Tatis is 22 years old.  He plays for the San Diego Padres in the NL.  His 162 game averages are 713 plate appearances and an ISP of 0.271
#####  Christian Yelich is 29 years old.  He plays for the Milwaukee Brewers in the NL.  His 162 game averages are 711 plate appearances and an ISP of 0.192
#####  Jose Ramirez will be 28 years old.  He plays for the Cleveland Indians in the AL.  His 162 game averages are 664 plate appearances and an ISP of 0.213
#####  Juan Soto is 22 years old.  He plays for the Washington Nationals in the NL.  His 162 game averages are 698 plate appearances and an ISP of 0.262
#####  Trea Turner is 27 years old.  He plays for the Washington Nationals in the NL.  His 162 game averages are 714 plate appearances and an ISP of 0.184
#####  Trevor Story is 28 years old.  He plays for the Colorado Rockies in the NL.  His 162 game averages are 683 plate appearances and an ISP of 0.258

In [94]:
samplebbstats = [{'AGE': 31,
  'PA': 687,
  'ISP': 0.214,
  'lgID::AL': 0,
  'lgID::FL': 0,
  'lgID::NL': 1,
  'lgID::PL': 0,
  'lgID::UA': 0,
  'ERA::Dead Ball': 0,
  'ERA::Live Ball': 0,
  'ERA::Integration': 0,
  'ERA::Divisional': 0,
  'ERA::Steroid': 0,
  'ERA::Current': 1},{'AGE': 29,
  'PA': 712,
  'ISP': 0.278,
  'lgID::AL': 1,
  'lgID::FL': 0,
  'lgID::NL': 0,
  'lgID::PL': 0,
  'lgID::UA': 0,
  'ERA::Dead Ball': 0,
  'ERA::Live Ball': 0,
  'ERA::Integration': 0,
  'ERA::Divisional': 0,
  'ERA::Steroid': 0,
  'ERA::Current': 1},{'AGE': 28,
  'PA': 739,
  'ISP': 0.221,
  'lgID::AL': 0,
  'lgID::FL': 0,
  'lgID::NL': 1,
  'lgID::PL': 0,
  'lgID::UA': 0,
  'ERA::Dead Ball': 0,
  'ERA::Live Ball': 0,
  'ERA::Integration': 0,
  'ERA::Divisional': 0,
  'ERA::Steroid': 0,
  'ERA::Current': 1},{'AGE': 23,
  'PA': 727,
  'ISP': 0.257,
  'lgID::AL': 0,
  'lgID::FL': 0,
  'lgID::NL': 1,
  'lgID::PL': 0,
  'lgID::UA': 0,
  'ERA::Dead Ball': 0,
  'ERA::Live Ball': 0,
  'ERA::Integration': 0,
  'ERA::Divisional': 0,
  'ERA::Steroid': 0,
  'ERA::Current': 1},{'AGE': 22,
  'PA': 713,
  'ISP': 0.271,
  'lgID::AL': 0,
  'lgID::FL': 0,
  'lgID::NL': 1,
  'lgID::PL': 0,
  'lgID::UA': 0,
  'ERA::Dead Ball': 0,
  'ERA::Live Ball': 0,
  'ERA::Integration': 0,
  'ERA::Divisional': 0,
  'ERA::Steroid': 0,
  'ERA::Current': 1},{'AGE': 29,
  'PA': 711,
  'ISP': 0.192,
  'lgID::AL': 0,
  'lgID::FL': 0,
  'lgID::NL': 1,
  'lgID::PL': 0,
  'lgID::UA': 0,
  'ERA::Dead Ball': 0,
  'ERA::Live Ball': 0,
  'ERA::Integration': 0,
  'ERA::Divisional': 0,
  'ERA::Steroid': 0,
  'ERA::Current': 1},{'AGE': 28,
  'PA': 664,
  'ISP': 0.213,
  'lgID::AL': 1,
  'lgID::FL': 0,
  'lgID::NL': 0,
  'lgID::PL': 0,
  'lgID::UA': 0,
  'ERA::Dead Ball': 0,
  'ERA::Live Ball': 0,
  'ERA::Integration': 0,
  'ERA::Divisional': 0,
  'ERA::Steroid': 0,
  'ERA::Current': 1},{'AGE': 22,
  'PA': 698,
  'ISP': 0.262,
  'lgID::AL': 0,
  'lgID::FL': 0,
  'lgID::NL': 1,
  'lgID::PL': 0,
  'lgID::UA': 0,
  'ERA::Dead Ball': 0,
  'ERA::Live Ball': 0,
  'ERA::Integration': 0,
  'ERA::Divisional': 0,
  'ERA::Steroid': 0,
  'ERA::Current': 1},{'AGE': 27,
  'PA': 714,
  'ISP': 0.184,
  'lgID::AL': 0,
  'lgID::FL': 0,
  'lgID::NL': 1,
  'lgID::PL': 0,
  'lgID::UA': 0,
  'ERA::Dead Ball': 0,
  'ERA::Live Ball': 0,
  'ERA::Integration': 0,
  'ERA::Divisional': 0,
  'ERA::Steroid': 0,
  'ERA::Current': 1},{'AGE': 28,
  'PA': 683,
  'ISP': 0.258,
  'lgID::AL': 0,
  'lgID::FL': 0,
  'lgID::NL': 1,
  'lgID::PL': 0,
  'lgID::UA': 0,
  'ERA::Dead Ball': 0,
  'ERA::Live Ball': 0,
  'ERA::Integration': 0,
  'ERA::Divisional': 0,
  'ERA::Steroid': 0,
  'ERA::Current': 1}]

In [95]:
pdSamplebbstats = pd.DataFrame.from_dict(samplebbstats)
pdSamplebbstats

,AGE,PA,ISP,lgID::AL,lgID::FL,lgID::NL,lgID::PL,lgID::UA,ERA::Dead Ball,ERA::Live Ball,ERA::Integration,ERA::Divisional,ERA::Steroid,ERA::Current
0,31,687,0.214,0,0,1,0,0,0,0,0,0,0,1
1,29,712,0.278,1,0,0,0,0,0,0,0,0,0,1
2,28,739,0.221,0,0,1,0,0,0,0,0,0,0,1
3,23,727,0.257,0,0,1,0,0,0,0,0,0,0,1
4,22,713,0.271,0,0,1,0,0,0,0,0,0,0,1
5,29,711,0.192,0,0,1,0,0,0,0,0,0,0,1
6,28,664,0.213,1,0,0,0,0,0,0,0,0,0,1
7,22,698,0.262,0,0,1,0,0,0,0,0,0,0,1
8,27,714,0.184,0,0,1,0,0,0,0,0,0,0,1
9,28,683,0.258,0,0,1,0,0,0,0,0,0,0,1


In [96]:
predictions = lr.predict(pdSamplebbstats)
predictions

array([0.86858484, 0.95871517, 0.88223264, 0.92601972, 0.94300606,
       0.83715433, 0.85528125, 0.92765095, 0.82308959, 0.92805439])

In [97]:
pdPredictedbbstats = pdSamplebbstats.copy()
pdPredictedbbstats['Predicted OPS'] = predictions

In [98]:
#Include player names and sort by predicted OPS
pdPredictedbbstats['Player'] = ['Freddie Freeman','Mike Trout','Mookie Betts','Ronald Acuna','Fernando Tatis','Christian Yelich','Jose Ramirez','Juan Soto','Trea Turner','Trevor Story']
pdPredicted_sort = pdPredictedbbstats.sort_values('Predicted OPS', ascending=False)
pdPredicted_sort

,AGE,PA,ISP,lgID::AL,lgID::FL,lgID::NL,lgID::PL,lgID::UA,ERA::Dead Ball,ERA::Live Ball,ERA::Integration,ERA::Divisional,ERA::Steroid,ERA::Current,Predicted OPS,Player
1,29,712,0.278,1,0,0,0,0,0,0,0,0,0,1,0.958715,Mike Trout
4,22,713,0.271,0,0,1,0,0,0,0,0,0,0,1,0.943006,Fernando Tatis
9,28,683,0.258,0,0,1,0,0,0,0,0,0,0,1,0.928054,Trevor Story
7,22,698,0.262,0,0,1,0,0,0,0,0,0,0,1,0.927651,Juan Soto
3,23,727,0.257,0,0,1,0,0,0,0,0,0,0,1,0.926020,Ronald Acuna
2,28,739,0.221,0,0,1,0,0,0,0,0,0,0,1,0.882233,Mookie Betts
0,31,687,0.214,0,0,1,0,0,0,0,0,0,0,1,0.868585,Freddie Freeman
6,28,664,0.213,1,0,0,0,0,0,0,0,0,0,1,0.855281,Jose Ramirez
5,29,711,0.192,0,0,1,0,0,0,0,0,0,0,1,0.837154,Christian Yelich
8,27,714,0.184,0,0,1,0,0,0,0,0,0,0,1,0.823090,Trea Turner


In [99]:
print(f"The projected top players in 2021 using OPS are {pdPredicted_sort[['Player', 'Predicted OPS']]}")

The projected top players in 2021 using OPS are              Player  Predicted OPS
1        Mike Trout       0.958715
4    Fernando Tatis       0.943006
9      Trevor Story       0.928054
7         Juan Soto       0.927651
3      Ronald Acuna       0.926020
2      Mookie Betts       0.882233
0   Freddie Freeman       0.868585
6      Jose Ramirez       0.855281
5  Christian Yelich       0.837154
8       Trea Turner       0.823090
